## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

### LDA on a real world dataset ###

We will be using the 20 Newsgroup dataset wich is a collection of emails exchanged in 20 newsgroups. You can read more about the dataset [here](http://qwone.com/~jason/20Newsgroups/).

## Step 1: Load the dataset ##

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: 
one for training (or development) and the other one for testing (or for performance evaluation). Some of these topics are similar, so we will try to classify them into 10 topics.

The split between the train and test set is based upon a messages posted before and after a specific date.

Remove all email headers, footers and quotes from the replied emails.

In [1]:
'''
Dataset from http://scikit-learn.org/stable/datasets/twenty_newsgroups.html
'''
from sklearn.datasets import fetch_20newsgroups
documents = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'), random_state=101).data

In [2]:
'''
Get the total number of documents
'''
print(len(documents))

11314


In [3]:
# Previewing a document
document_num = 4310
print("\n**Printing out a sample document:**")
print(documents[document_num])


**Printing out a sample document:**


Fret not, you made it.


Not while we still have our guns.  <evil grin>  

Hey, gang, it's not about duck hunting, or about dark alleys,
it's about black-clad, helmeted and booted troops storming
houses and violating civil rights under color of law. 

Are YOU ready to defend YOUR Constitution?


** Looks like this document will come under `Politics`. **

## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

In [4]:
'''
Loading Gensim library
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [5]:
'''
Lemmatizing example for a verb, noun.
'''
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense
print(WordNetLemmatizer().lemmatize('Churches', pos = 'n')) # noun

go
Churches


In [6]:
'''
Stemming example
'''
stemmer = SnowballStemmer("english")
plurals = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in plurals]
print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [7]:
'''
Data preprocessing
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result = [lemmatize_stemming(token) for token in gensim.utils.simple_preprocess(text) 
              # Remove stop words and words less than 3 characters long
              if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3]
    return result

In [8]:
'''
Preview a document after preprocessing
'''
print("Original document: ")
words = []
for word in documents[document_num].split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(documents[document_num]))

Original document: 
['\n\nFret', 'not,', 'you', 'made', 'it.\n\n\nNot', 'while', 'we', 'still', 'have', 'our', 'guns.', '', '<evil', 'grin>', '', '\n\nHey,', 'gang,', "it's", 'not', 'about', 'duck', 'hunting,', 'or', 'about', 'dark', "alleys,\nit's", 'about', 'black-clad,', 'helmeted', 'and', 'booted', 'troops', 'storming\nhouses', 'and', 'violating', 'civil', 'rights', 'under', 'color', 'of', 'law.', '\n\nAre', 'YOU', 'ready', 'to', 'defend', 'YOUR', 'Constitution?']


Tokenized and lemmatized document: 
['fret', 'gun', 'evil', 'grin', 'gang', 'duck', 'hunt', 'dark', 'alley', 'black', 'cloth', 'helmet', 'boot', 'troop', 'storm', 'hous', 'violat', 'civil', 'right', 'color', 'readi', 'defend', 'constitut']


In [9]:
'''
Perform preprocessing on entire dataset
'''
processed_docs = [preprocess(doc) for doc in documents]

## Step 3.1: Bag of words on the dataset ##

In [10]:
'''
Create a dictionary containing the number of times a word appears in the training set
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [11]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 snip
1 place
2 death
3 soldier
4 patrol
5 occupi
6 lebanes
7 terrritori
8 terror
9 murder
10 disingen


** Gensim filter_extremes **

`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [12]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, 
                                no_above=0.10)

** Gensim doc2bow **

`doc2bow(document)`

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [13]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear.
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [14]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
# document_num = 4310
bow_corpus[document_num]

[(146, 1),
 (195, 1),
 (199, 1),
 (200, 1),
 (201, 1),
 (202, 1),
 (203, 1),
 (204, 1),
 (205, 1),
 (206, 1),
 (207, 1),
 (208, 1),
 (209, 1),
 (210, 1),
 (211, 1),
 (212, 1),
 (213, 1),
 (214, 1),
 (229, 1),
 (4202, 1)]

In [15]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 146 ("color") appears 1 time.
Word 195 ("evil") appears 1 time.
Word 199 ("gang") appears 1 time.
Word 200 ("duck") appears 1 time.
Word 201 ("hunt") appears 1 time.
Word 202 ("dark") appears 1 time.
Word 203 ("black") appears 1 time.
Word 204 ("cloth") appears 1 time.
Word 205 ("helmet") appears 1 time.
Word 206 ("boot") appears 1 time.
Word 207 ("troop") appears 1 time.
Word 208 ("storm") appears 1 time.
Word 209 ("hous") appears 1 time.
Word 210 ("violat") appears 1 time.
Word 211 ("civil") appears 1 time.
Word 212 ("readi") appears 1 time.
Word 213 ("defend") appears 1 time.
Word 214 ("constitut") appears 1 time.
Word 229 ("gun") appears 1 time.
Word 4202 ("grin") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [16]:
'''
Create tf-idf model object on 
'''
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [17]:
'''
Apply transformation to the entire corpus
'''
corpus_tfidf = tfidf[bow_corpus]

In [18]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
for doc in corpus_tfidf:
    pprint(doc)
    break

NameError: name 'pprint' is not defined

## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)

Alpha is the per document topic distribution.
* High alpha: Every document has a mixture of all topics(documents appear similar to each other).
* Low alpha: Every document has a mixture of very few topics

Eta is the per topic word distribution.
* High eta: Each topic has a mixture of most words(topics appear similar to each other).
* Low eta: Each topic has a mixture of few words.

In [ ]:
# LDA mono-core
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,
#                                    iterations = 10, 
#                                    passes = 1)

# LDA multicore 
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 50, 
                                       workers=2)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

### Classification of the topics ###

Using the words in each topic and their corresponding weights, we have the following categories:

* 0: Computer Software
* 1: Space
* 2: Government
* 3: Energy
* 4: Sports
* 5: Applications
* 6: Religion
* 7: Computer Hardware
* 8: Automotives
* 9: Politics

## Step 4.2 Running LDA using TF-IDF ##

In [ ]:
'''
Define lda model using tfidf corpus
'''
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 50, 
                                             workers=4)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, we have the following categories:

* 0: For sale
* 1: Space
* 2: Politics
* 3: Religion
* 4: Sports
* 5: Automotives
* 6: Computer Software
* 7: Computer Hardware
* 8: Unclear
* 9: Sports - Hockey

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [ ]:
'''
Text of sample document 4310
'''
print(documents[document_num])

In [ ]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

### It has the highest probability (`64.5%`) to be  part of the topic that we assigned as Politics, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [ ]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

### It has the highest probability (`71.4%`) to be  part of the topic that we assigned as Politics, which is the accurate classification. ###

## Step 5.2: Performance evaluation of model as a whole ##

NOTE: log perplexity unclear

We use the perplexity(logarithm) for our evaluation. The perplexity is connected to the log likelihood that the model is able to **generate the documents, given the distribution of topics for those documents**. The lower the perplexity, the better the model as it signifies that the model can regenerate the text quite well.

In [ ]:
'''
Calcualte log perplexity of the model to check if the model can generate the text.
'''
print("Log perplexity of the model is", lda_model.log_perplexity(bow_corpus))

## Step 6: Testing model on unseen document ##

In [ ]:
unseen_document = "My favorite activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

The model correctly classifies the unseen document with 61.6% probability to the Sports category.